# Neural Network Models

</br>

### Imports and Setup

In [14]:
import pandas as pd
import numpy as np
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

# Deep Learning imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, regularizers
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# print(f"TensorFlow version: {tf.__version__}")
# print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Load data
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')

# Load metadata and tree model results
with open('preprocessing_metadata.json', 'r') as f:
    metadata = json.load(f)

with open('tree_model_summary.pkl', 'rb') as f:
    tree_results = pickle.load(f)

print("\n" + "="*50)
print("NEURAL NETWORK MODELS FOR PORE PRESSURE")
print("="*50)
print(f"Baseline to beat: CatBoost R² = {tree_results['best_r2']:.4f}")
print(f"Best target from trees: {tree_results['best_target']}")
print(f"Train samples: {len(train_df):,}")
print(f"Val samples: {len(val_df):,}")


NEURAL NETWORK MODELS FOR PORE PRESSURE
Baseline to beat: CatBoost R² = 0.6132
Best target from trees: overpressure
Train samples: 153,638
Val samples: 28,600


### Data Preparation for Neural Networks

In [15]:
print("="*50)
print("DATA PREPARATION FOR NEURAL NETWORKS")
print("="*50)

# Get feature columns (excluding targets)
feature_cols = [f for f in metadata['predictor_features'] 
                if f not in ['ppp', 'pressure_ratio', 'overpressure']]

# Prepare features and targets
X_train = train_df[feature_cols].values
X_val = val_df[feature_cols].values

# Try overpressure target (best from tree models)
y_train_overpressure = (train_df['ppp'] - train_df['hp']).values
y_val_overpressure = (val_df['ppp'] - val_df['hp']).values

# Also prepare log(ppp) for comparison
y_train_log = np.log1p(train_df['ppp'].values)
y_val_log = np.log1p(val_df['ppp'].values)

print(f"Feature shape: {X_train.shape[1]} features")
print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")

# Feature scaling - critical for neural networks
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Target scaling for overpressure
target_scaler_op = StandardScaler()
y_train_op_scaled = target_scaler_op.fit_transform(y_train_overpressure.reshape(-1, 1)).flatten()
y_val_op_scaled = target_scaler_op.transform(y_val_overpressure.reshape(-1, 1)).flatten()

print(f"\nTarget statistics (overpressure):")
print(f"  Train: {y_train_overpressure.mean():.0f} ± {y_train_overpressure.std():.0f} psi")
print(f"  Val: {y_val_overpressure.mean():.0f} ± {y_val_overpressure.std():.0f} psi")

# Check for pressure regimes
pressure_ratio_train = train_df['ppp'] / train_df['hp']
pressure_ratio_val = val_df['ppp'] / val_df['hp']

print(f"\nPressure regime distribution:")
print(f"  Train - Under: {(pressure_ratio_train < 0.9).mean()*100:.1f}%")
print(f"  Train - Normal: {((pressure_ratio_train >= 0.9) & (pressure_ratio_train <= 1.1)).mean()*100:.1f}%")
print(f"  Train - Over: {(pressure_ratio_train > 1.1).mean()*100:.1f}%")

DATA PREPARATION FOR NEURAL NETWORKS
Feature shape: 15 features
Training samples: 153638
Validation samples: 28600

Target statistics (overpressure):
  Train: 758 ± 987 psi
  Val: 330 ± 527 psi

Pressure regime distribution:
  Train - Under: 13.8%
  Train - Normal: 21.8%
  Train - Over: 64.4%


### Build Deep Feedforward Neural Network (DFNN)

In [16]:
print("="*50)
print("DEEP FEEDFORWARD NEURAL NETWORK (DFNN)")
print("="*50)

def create_dfnn(input_dim, hidden_layers=[256, 128, 64, 32], 
                dropout_rate=0.3, l2_reg=0.01):
    """Create a deep feedforward network"""
    
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.BatchNormalization()
    ])
    
    # Hidden layers with decreasing neurons
    for i, units in enumerate(hidden_layers):
        model.add(layers.Dense(
            units, 
            activation='relu',
            kernel_regularizer=regularizers.l2(l2_reg),
            kernel_initializer='he_normal'
        ))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))
    
    # Output layer
    model.add(layers.Dense(1, activation='linear'))
    
    return model

# Create model
dfnn_model = create_dfnn(
    input_dim=X_train_scaled.shape[1],
    hidden_layers=[512, 256, 128, 64, 32],
    dropout_rate=0.3,
    l2_reg=0.001
)

# Compile
dfnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print(dfnn_model.summary())

# Callbacks
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-6
)

# Train
print("\nTraining DFNN...")
history = dfnn_model.fit(
    X_train_scaled, y_train_op_scaled,
    validation_data=(X_val_scaled, y_val_op_scaled),
    epochs=200,
    batch_size=256,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Evaluate
y_pred_scaled = dfnn_model.predict(X_val_scaled, verbose=0)
y_pred_op = target_scaler_op.inverse_transform(y_pred_scaled)
y_val_op = y_val_overpressure

# Convert to PPP for evaluation
ppp_pred = y_pred_op.flatten() + val_df['hp'].values
ppp_true = val_df['ppp'].values

dfnn_r2 = r2_score(ppp_true, ppp_pred)
dfnn_rmse = np.sqrt(mean_squared_error(ppp_true, ppp_pred))

print(f"\nDFNN Results:")
print(f"  R²: {dfnn_r2:.4f}")
print(f"  RMSE: {dfnn_rmse:.1f} psi")

# Check underpressure performance
under_mask = pressure_ratio_val < 0.9
if under_mask.sum() > 0:
    under_r2 = r2_score(ppp_true[under_mask], ppp_pred[under_mask])
    print(f"  Underpressure R²: {under_r2:.4f}")

print(f"\nComparison:")
print(f"  CatBoost baseline: R² = {tree_results['best_r2']:.4f}")
print(f"  DFNN: R² = {dfnn_r2:.4f}")
print(f"  Improvement: {(dfnn_r2 - tree_results['best_r2'])*100:+.1f}%")

DEEP FEEDFORWARD NEURAL NETWORK (DFNN)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization_22          │ (None, 15)             │            60 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 512)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 186,813 (729.74 KB)

 Trainable params: 184,799 (721.87 KB)

 Non-trainable params: 2,014 (7.87 KB)

None

Training DFNN...
Epoch 1/200
601/601 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 2.4523 - mae: 0.5966 - val_loss: 1.7234 - val_mae: 0.3380 - learning_rate: 0.0010
Epoch 2/200
601/601 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 1.4774 - mae: 0.3942 - val_loss: 1.1329 - val_mae: 0.3401 - learning_rate: 0.0010
Epoch 3/200
601/601 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.9115 - mae: 0.3652 - val_loss: 0.6934 - val_mae: 0.3419 - learning_rate: 0.0010
Epoch 4/200
601/601 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.5517 - mae: 0.3513 - val_loss: 0.4588 - val_mae: 0.3595 - learning_rate: 0.0010
Epoch 5/200
601/601 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.3638 - mae: 0.3440 - val_loss: 0.3757 - val_mae: 0.3896 - learning_rate: 0.0010
Epoch 6/200
601/601 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.2808 - mae: 0.3390 - val_loss: 0.3780 - val_mae: 0.4361 - learning_rate: 0.0010
Epoch 7/200
601/601 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.2468 - mae: 0.3347 - val_loss: 0.3959 - val_mae: 0.4760 

In [17]:
print("="*50)
print("DFNN IMPROVEMENTS - LAST ATTEMPT")
print("="*50)

# 1. Add engineered features for DFNN
def add_dfnn_features(df):
    """Add features that help DFNN without temporal leakage"""
    df_feat = df.copy()
    
    # Depth-based polynomial features
    df_feat['tvd_squared'] = df_feat['tvd'] ** 2
    df_feat['tvd_log'] = np.log1p(df_feat['tvd'])
    
    # Interaction terms (domain knowledge)
    df_feat['eaton_tvd'] = df_feat['eaton_ratio'] * df_feat['tvd_normalized']
    df_feat['hp_ob_ratio'] = df_feat['hp'] / (df_feat['ob'] + 1e-6)
    
    # Binned depth regions (helps DFNN learn different behaviors at different depths)
    df_feat['depth_bin'] = pd.cut(df_feat['tvd'], bins=10, labels=False)
    
    return df_feat

# Apply feature engineering
train_enhanced = add_dfnn_features(train_df)
val_enhanced = add_dfnn_features(val_df)

# 2. Try ensemble of DFNNs with different targets
def create_specialized_dfnn(input_dim, architecture='deep'):
    """Different DFNN architectures"""
    
    if architecture == 'deep':
        # Very deep network
        hidden = [1024, 512, 256, 128, 64, 32, 16]
    elif architecture == 'wide':
        # Wide but shallow
        hidden = [2048, 1024, 512]
    else:  # 'funnel'
        # Aggressive funnel
        hidden = [512, 128, 32, 8]
    
    model = models.Sequential([layers.Input(shape=(input_dim,))])
    
    for units in hidden:
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(1, activation='linear'))
    return model

# 3. Well-specific normalization (key insight from CatBoost)
from sklearn.preprocessing import LabelEncoder

# Encode wells
le = LabelEncoder()
train_enhanced['well_encoded'] = le.fit_transform(train_enhanced['well_id'])
val_enhanced['well_encoded'] = le.transform(val_enhanced['well_id'])

# Add well as one-hot encoded features
well_one_hot_train = pd.get_dummies(train_enhanced['well_encoded'], prefix='well')
well_one_hot_val = pd.get_dummies(val_enhanced['well_encoded'], prefix='well')

# Combine features
feature_cols_enhanced = feature_cols + ['tvd_squared', 'tvd_log', 'eaton_tvd', 
                                        'hp_ob_ratio', 'depth_bin']
X_train_enhanced = pd.concat([
    train_enhanced[feature_cols_enhanced],
    well_one_hot_train
], axis=1).values

X_val_enhanced = pd.concat([
    val_enhanced[feature_cols_enhanced],
    well_one_hot_val
], axis=1).values

# Scale
scaler_enhanced = StandardScaler()
X_train_scaled = scaler_enhanced.fit_transform(X_train_enhanced)
X_val_scaled = scaler_enhanced.transform(X_val_enhanced)

# Train improved DFNN
print(f"Enhanced features: {X_train_scaled.shape[1]}")

dfnn_final = create_specialized_dfnn(X_train_scaled.shape[1], 'deep')
dfnn_final.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='huber',  # More robust to outliers
    metrics=['mae']
)

# Use overpressure target
y_train = train_enhanced['ppp'].values - train_enhanced['hp'].values
y_val = val_enhanced['ppp'].values - val_enhanced['hp'].values

# Train with better strategy
history = dfnn_final.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=200,
    batch_size=512,  # Larger batch for stability
    callbacks=[
        callbacks.EarlyStopping(patience=30, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(factor=0.5, patience=10)
    ],
    verbose=1
)

# Evaluate
y_pred = dfnn_final.predict(X_val_scaled, verbose=0).flatten()
ppp_pred = y_pred + val_enhanced['hp'].values
ppp_true = val_enhanced['ppp'].values

final_r2 = r2_score(ppp_true, ppp_pred)
print(f"\nImproved DFNN R²: {final_r2:.4f}")
print(f"Original DFNN R²: 0.5458")
print(f"CatBoost baseline: 0.6132")

DFNN IMPROVEMENTS - LAST ATTEMPT


ValueError: y contains previously unseen labels: 'Balkassar OXY 01'